In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/dacon/lowresol/

In [ ]:
# !unzip -qn open.zip -d ./open/

In [1]:
!pip install --quiet timm pytorch_lightning==1.7.7 torchmetrics==0.11.1

DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import os
import pickle
import gc
import warnings
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pytorch_lightning as L
from torch import optim

from torchinfo import summary
#from glob import glob
import glob
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torchvision.transforms import v2 as  transforms
from torch.utils.data import Dataset, DataLoader
from transformers import Swinv2Config, Swinv2Model, AutoImageProcessor, AutoModelForImageClassification
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from pytorch_lightning.loggers import WandbLogger  # wandb logger를 임포트


c:\Users\Seo\anaconda3\envs\Dron_PytorchLightning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    NUM_DEVICES = torch.cuda.device_count()
    NUM_WORKERS = os.cpu_count()
    #NUM_CLASSES = 4
    NUM_CLASSES = 8
    EPOCHS = 16
    # BATCH_SIZE = (
    #     32 if torch.cuda.device_count() < 2 
    #     else (32 * torch.cuda.device_count())
    # )
    BATCH_SIZE = 16
    LR = 0.001
    APPLY_SHUFFLE = True
    SEED = 768
    #HEIGHT = 224
    #WIDTH = 224
    HEIGHT = 224
    WIDTH = 224
    CHANNELS = 3
    #IMAGE_SIZE = (224, 224, 3)
    IMAGE_SIZE = (224, 224, 3)
    
    # Define paths
    #DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/dataset"
    #TRAIN_PATH = '/content/drive/MyDrive/Colab Notebooks/dataset/train/'
    #TEST_PATH = '/content/drive/MyDrive/Colab Notebooks/dataset/test'
    
    # Define paths
    # DATASET_PATH = "C:\\Users\\Seo\\Desktop\\2024_06_Dron_Competition\\Data"
    # TRAIN_PATH = 'C:\\Users\\Seo\\Desktop\\2024_06_Dron_Competition\\Data\\train\\'
    # TEST_PATH = 'C:\\Users\\Seo\\Desktop\\2024_06_Dron_Competition\\Data\\test\\'

    # Define paths
    DATASET_PATH = "C:\\Users\\Seo\\Desktop\\Gits\\2024_06_Dron_Competition\\Datas_3000_3000"
    TRAIN_PATH = 'C:\\Users\\Seo\\Desktop\\Gits\\2024_06_Dron_Competition\\Datas_3000_3000\\'
    TEST_PATH = 'C:\\Users\\Seo\\Desktop\\Gits\\2024_06_Dron_Competition\\Datas_3000_2250\\'
    
    
# Mute warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [3]:
save_dir = 'saved'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

result_dir = 'result'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

<a id='1.1'></a>
### Get image paths with glob

In [4]:
#%%time
#print(f"{CFG.TRAIN_PATH}**/*.jpg")
train_images = glob.glob(f"{CFG.TRAIN_PATH}**/*.jpg")
valid_images = glob.glob(f"{CFG.TEST_PATH}**/*.jpg")

<a id='1.2'></a>
### Create Pandas DataFrames for paths and labels

In [5]:
def generate_labels(image_paths):
    return [(_.split('\\')[-2:][0]).replace('-', '_') for _ in image_paths]


def build_df(image_paths, labels):
    # Create dataframe
    df = pd.DataFrame({
        'image_path': image_paths,
        'label': generate_labels(labels)
    })
    
    # Return df
    return df

In [6]:
train_no_kfold_df = build_df(train_images, train_images)
valid_no_kfold_df = build_df(valid_images, valid_images)

# View first 5 samples in the dataset
train_no_kfold_df.head(5)

,image_path,label
0,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono
1,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono
2,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono
3,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono
4,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono


In [7]:
valid_no_kfold_df.head(5)

,image_path,label
0,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono
1,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono
2,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono
3,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono
4,C:\Users\Seo\Desktop\Gits\2024_06_Dron_Competi...,Bonobono


In [8]:
train_no_kfold_df.shape, valid_no_kfold_df.shape

((2424, 2), (2424, 2))

<a id='1.3'></a>
### Load & View Random Sample Image

In [9]:
from PIL import Image
import matplotlib.pyplot as plt

In [10]:
def _load(image_path, as_tensor=True):
    # Read and decode an image file to a uint8 tensor
    image = Image.open(image_path)
    
    if as_tensor:
        converter = transforms.Compose([
            transforms.ToTensor(), 
            #transforms.Grayscale()
        ])
        return converter(image)
    else:
        return image


def view_sample(image, label, color_map='rgb', fig_size=(8, 10)):
    plt.figure(figsize=fig_size)
    
    if color_map=='rgb':
        plt.imshow(image)
    else:
        plt.imshow(image, cmap=color_map)
    
    plt.title(f'Label: {label}', fontsize=16)
    return

<a id='1.5'></a>
### Create Train and Validation sets

In [11]:
class CustomDataset(Dataset):
    def __init__(self, df, path_col,  mode='train'):
        self.df = df
        self.path_col = path_col
        self.mode = mode

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.mode == 'train':
            row = self.df.iloc[idx]
            image = read_image(row[self.path_col])/256.
            label = row['class']
            data = {
                'image':image,
                'label':label
            }
            return data
        elif self.mode == 'val':
            row = self.df.iloc[idx]
            image = read_image(row[self.path_col])/256.
            label = row['class']
            data = {
                'image':image,
                'label':label
            }
            return data
        elif self.mode == 'inference':
            row = self.df.iloc[idx]
            image = read_image(row[self.path_col])/256.
            data = {
                'image':image,
            }
            return data

    def train_transform(self, image):
        pass

In [12]:
class CustomCollateFn:
    def __init__(self, transform, mode):
        self.mode = mode
        self.transform = transform

    def __call__(self, batch):
        if self.mode=='train':
            images = torch.stack([self.transform(data['image']) for data in batch])
            labels = torch.LongTensor([data['label'] for data in batch])
            return images, labels
        elif self.mode=='val':
            images = torch.stack([self.transform(data['image']) for data in batch])
            labels = torch.LongTensor([data['label'] for data in batch])
            return images, labels
        elif self.mode=='inference':
            images = torch.stack([self.transform(data['image']) for data in batch])
            return images, 

In [13]:
SEED = 42
N_SPLIT = 5

In [14]:
L.seed_everything(SEED)

Global seed set to 42


42

In [15]:
def create_or_load_class_mapping(df, pickle_file_path):
    # 미리 정의된 클래스 매핑
    class_mapping = {
        'Bonobono': 0,
        'Eagle': 1,
        'Gmarket': 2,
        'Hp': 3,
        'Intel': 4,
        'underwood_statue': 5,
        'Wonju': 6,
        'Yonsei': 7
    }
    
    # pickle 파일이 존재하지 않으면 생성
    if not os.path.exists(pickle_file_path):
        print("Pickle 파일을 생성합니다.")
        df['class'] = df['label'].map(class_mapping)
        
        with open(pickle_file_path, 'wb') as f:
            pickle.dump(class_mapping, f)
        
        return class_mapping
    else:
        # pickle 파일이 존재하면 로드
        print("Pickle 파일을 로드합니다.")
        with open(pickle_file_path, 'rb') as f:
            class_mapping = pickle.load(f)
        
        return class_mapping

In [16]:
# 함수를 호출하여 class_mapping 생성 또는 로드
pickle_file_path = 'C:\\Users\\Seo\\Desktop\\Gits\\2024_06_Dron_Competition\\class8.pickle'
class_mapping = create_or_load_class_mapping(train_no_kfold_df, pickle_file_path)

Pickle 파일을 로드합니다.


In [17]:
# 클래스 매핑을 이용하여 라벨을 클래스 번호로 변환 (예시로 train_no_kfold_df 사용)
train_no_kfold_df['class'] = train_no_kfold_df['label'].apply(lambda x: class_mapping.get(x))
valid_no_kfold_df['class'] = valid_no_kfold_df['label'].apply(lambda x: class_mapping.get(x))

In [18]:
le_no_kfold = LabelEncoder()
train_no_kfold_df['class'] = le_no_kfold.fit_transform(train_no_kfold_df['label'])
valid_no_kfold_df['class'] = le_no_kfold.fit_transform(valid_no_kfold_df['label'])

In [19]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

# 예시 데이터프레임 생성
# valid_no_kfold_df = pd.DataFrame({
#     'label': [...],
#     'class': [...]
# })

# 샘플링할 데이터 비율 (예: 전체 데이터의 20%)
sample_ratio = 0.4

# StratifiedShuffleSplit 객체 생성
splitter = StratifiedShuffleSplit(n_splits=1, test_size=sample_ratio, random_state=42)

# 샘플링
for train_index, sample_index in splitter.split(valid_no_kfold_df, valid_no_kfold_df['class']):
    sampled_df = valid_no_kfold_df.iloc[sample_index]

valid_no_kfold_df = sampled_df

In [20]:
#if not len(train_df) == len(os.listdir('./open/train')):
#    raise ValueError()

In [20]:
from torchvision.transforms import v2 as  transforms
from torchvision.transforms import RandomAffine, RandomHorizontalFlip, RandomVerticalFlip, ColorJitter

def add_random_noise(image):
    noise = torch.randn_like(image) * 0.1
    return image + noise

train_transform = transforms.Compose([
    transforms.Resize(size=(CFG.WIDTH,CFG.WIDTH), interpolation=transforms.InterpolationMode.BICUBIC),
    RandomAffine(degrees=30, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value='random'), 
    RandomHorizontalFlip(p=0.5),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Lambda(add_random_noise),
    # transforms.RandomGrayscale(p=0.1),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

val_transform = transforms.Compose([
    transforms.Resize(size=(CFG.WIDTH,CFG.WIDTH), interpolation=transforms.InterpolationMode.BICUBIC),
    RandomAffine(degrees=30, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    RandomHorizontalFlip(p=0.5),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Lambda(add_random_noise),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

train_collate_fn = CustomCollateFn(train_transform, 'train')
val_collate_fn = CustomCollateFn(val_transform, 'val')

In [21]:
class EfficientNetV2Model(nn.Module):
    def __init__(self, backbone_model, name='efficientnet-v2-large', 
                 num_classes=CFG.NUM_CLASSES, device=CFG.DEVICE):
        super(EfficientNetV2Model, self).__init__()
        
        self.backbone_model = backbone_model
        self.device = device
        self.num_classes = num_classes
        self.name = name
        
        classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=0.2, inplace=True), 
            nn.Linear(in_features=1280, out_features=256, bias=True),
            nn.GELU(),
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(in_features=256, out_features=num_classes, bias=False)
        ).to(device)
        
        self._set_classifier(classifier)
        
    def _set_classifier(self, classifier:nn.Module) -> None:
        self.backbone_model.classifier = classifier
    
    def forward(self, image):
        return self.backbone_model(image)

In [22]:
def get_effiecientnetv2_model(
    device: torch.device=CFG.NUM_CLASSES) -> nn.Module:
    # Set the manual seeds
    torch.manual_seed(CFG.SEED)
    torch.cuda.manual_seed(CFG.SEED)

    # Get model weights
    model_weights = (
        torchvision
        .models
        .EfficientNet_V2_L_Weights
        .DEFAULT
    )
    
    # Get model and push to device
    model = (
        torchvision.models.efficientnet_v2_l(
            weights=model_weights
        )
    ).to(device) 
    
    # Freeze Model Parameters
    for param in model.features.parameters():
        param.requires_grad = False
        
    return model

In [23]:
# Get EfficientNet v2 model
backbone_model = get_effiecientnetv2_model(CFG.DEVICE)

In [24]:
efficientnetv2_params = {
    'backbone_model'    : backbone_model,
    'name'              : 'efficientnet-v2-large',
    'device'            : CFG.DEVICE
}

# Generate Model
efficientnet_model = EfficientNetV2Model(**efficientnetv2_params)

# If using GPU T4 x2 setup, use this:
if CFG.NUM_DEVICES > 1:
    efficientnet_model = nn.DataParallel(efficientnet_model)

In [25]:
# View model summary
summary(
    model=efficientnet_model, 
    input_size=(CFG.BATCH_SIZE, CFG.CHANNELS, CFG.WIDTH, CFG.HEIGHT),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                                           Input Shape          Output Shape         Param #              Trainable
EfficientNetV2Model (EfficientNetV2Model)                         [16, 3, 224, 224]    [16, 8]              --                   Partial
├─EfficientNet (backbone_model)                                   [16, 3, 224, 224]    [16, 8]              --                   Partial
│    └─Sequential (features)                                      [16, 3, 224, 224]    [16, 1280, 7, 7]     --                   False
│    │    └─Conv2dNormActivation (0)                              [16, 3, 224, 224]    [16, 32, 112, 112]   (928)                False
│    │    └─Sequential (1)                                        [16, 32, 112, 112]   [16, 32, 112, 112]   (37,120)             False
│    │    └─Sequential (2)                                        [16, 32, 112, 112]   [16, 64, 56, 56]     (1,032,320)          False
│    │    └─Sequential (3)                     

In [26]:
class Pytorch_CustomModel(nn.Module):
    def __init__(self):
        super(Pytorch_CustomModel, self).__init__()
        self.model = efficientnet_model
        self.clf = nn.Sequential(
            nn.SiLU(),
            nn.LazyLinear(CFG.NUM_CLASSES),
        )

#     @torch.compile
    def forward(self, x, label=None):
        # original
        x = self.model(x)
        # x = self.clf(x)
        loss = None
        if label is not None:
            loss = nn.CrossEntropyLoss()(x, label)
        probs = nn.LogSoftmax(dim=-1)(x)
        return probs, loss

In [27]:
def train(model, epoch, total_epoch, train_dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch}/{total_epoch}")
    for i, (data, target) in progress_bar:
        data, target = data.to(CFG.DEVICE), target.to(CFG.DEVICE)
        optimizer.zero_grad()
        output, loss_ = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

         # 손실 및 정확도 계산
        running_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
        # tqdm의 설명 부분을 업데이트
        progress_bar.set_postfix(loss=running_loss/(i+1), accuracy=100.*correct/total)
    # 에포크 완료 후 출력
    print(f"Epoch {epoch}, Loss: {running_loss/len(train_dataloader):.4f}, Accuracy: {correct/total:.4f}")

In [28]:
# 검증 함수
def valid(model, val_loader, criterion):
    model.eval()
    val_loss = 0
    correct = 0
    progress_bar = tqdm(val_loader, total=len(val_loader))
    with torch.no_grad():
        for data, target in progress_bar:
            data, target = data.to(CFG.DEVICE), target.to(CFG.DEVICE)
            output, loss_ = model(data)
            val_loss += criterion(output, target).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    val_loss /= len(val_loader.dataset)
    print('Valid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    #wandb.log({"Valid Accuracy": 100. * correct / len(val_loader.dataset), "Valid Loss": val_loss})

    return val_loss

In [29]:
train_fold_df = train_no_kfold_df
val_fold_df = valid_no_kfold_df

train_dataset = CustomDataset(train_fold_df, 'image_path', mode='train')
val_dataset = CustomDataset(val_fold_df, 'image_path', mode='val')

train_dataloader = DataLoader(train_dataset, collate_fn=train_collate_fn, batch_size=CFG.BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, collate_fn=val_collate_fn, batch_size=CFG.BATCH_SIZE)

# 모델, 손실함수, 최적화함수 설정
model = Pytorch_CustomModel().to(CFG.DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=1e-5, weight_decay=5e-4, eps=5e-9)

# wandb에 모델, 최적화 함수 로그
#wandb.watch(model, log="all")
#wandb.config.update({"Optimizer": "ADAM", "Learning Rate": 0.01, "Momentum": 0.5})

best_val_loss = float('inf')

for epoch in range(CFG.EPOCHS):
    
    train(model, epoch, CFG.EPOCHS, train_dataloader, criterion, optimizer)

    val_loss = valid(model, val_dataloader, criterion)
    checkpoint_dir = './checkpoints'
    
    os.makedirs(checkpoint_dir, exist_ok=True)
    # 모델 가중치 저장
    checkpoint_path = os.path.join(checkpoint_dir, f'EfficientNetV2_epoch{epoch}_valid_loss{val_loss}.pth')
    torch.save(model.state_dict(), checkpoint_path)
    # Validation 성능이 향상될 때마다 가장 좋은 모델 가중치 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_checkpoint_path = os.path.join(checkpoint_dir, f'EfficientNetV2_best_model_valid_loss{val_loss}.pth')
        torch.save(model.state_dict(), best_checkpoint_path)

c:\Users\Seo\anaconda3\envs\Dron_PytorchLightning\lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
Epoch 0/16: 100%|██████████| 152/152 [03:58<00:00,  1.57s/it, accuracy=25.7, loss=2]   


Epoch 0, Loss: 1.9991, Accuracy: 0.2566


100%|██████████| 61/61 [01:10<00:00,  1.16s/it]


Valid set: Average loss: 0.1216, Accuracy: 573/970 (59%)



Epoch 1/16: 100%|██████████| 152/152 [03:59<00:00,  1.57s/it, accuracy=31, loss=nan]  


Epoch 1, Loss: nan, Accuracy: 0.3098


100%|██████████| 61/61 [01:12<00:00,  1.18s/it]


Valid set: Average loss: nan, Accuracy: 126/970 (13%)



Epoch 2/16:  47%|████▋     | 72/152 [01:55<02:08,  1.61s/it, accuracy=11.6, loss=nan]


KeyboardInterrupt: 

In [32]:

train_fold_df = train_no_kfold_df
val_fold_df = valid_no_kfold_df

train_dataset = CustomDataset(train_fold_df, 'image_path', mode='train')
val_dataset = CustomDataset(val_fold_df, 'image_path', mode='val')

train_dataloader = DataLoader(train_dataset, collate_fn=train_collate_fn, batch_size=CFG.BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, collate_fn=val_collate_fn, batch_size=CFG.BATCH_SIZE)

model = Swinv2Model.from_pretrained("microsoft/swinv2-large-patch4-window12to16-192to256-22kto1k-ft")
#model = efficientnet_model

lit_model = LitCustomModel(model)

c:\Users\Seo\anaconda3\envs\Dron_PytorchLightning\lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [34]:
# View model summary
summary(
    model=lit_model, 
    input_size=(CFG.BATCH_SIZE, CFG.CHANNELS, CFG.WIDTH, CFG.HEIGHT),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                                                               Input Shape          Output Shape         Param #              Trainable
Swinv2Model (Swinv2Model)                                                             [32, 3, 224, 224]    [32, 1536]           --                   True
├─Swinv2Embeddings (embeddings)                                                       [32, 3, 224, 224]    [32, 3136, 192]      --                   True
│    └─Swinv2PatchEmbeddings (patch_embeddings)                                       [32, 3, 224, 224]    [32, 3136, 192]      --                   True
│    │    └─Conv2d (projection)                                                       [32, 3, 224, 224]    [32, 192, 56, 56]    9,408                True
│    └─LayerNorm (norm)                                                               [32, 3136, 192]      [32, 3136, 192]      384                  True
│    └─Dropout (dropout)                                               

In [35]:
# 체크포인트 파일 경로
ckpt_path = 'C:\\Users\\Seo\\Desktop\\2024_06_Dron_Competition\\checkpoints\\swinv2-large-resize-epoch=06-train_loss=0.0012-val_score=0.9991.ckpt'

# 체크포인트 로드
checkpoint = torch.load(ckpt_path)

# 모델 상태 로드
lit_model.load_state_dict(checkpoint['state_dict'])

# .pth 파일 경로
pth_path = 'C:\\Users\\Seo\\Desktop\\2024_06_Dron_Competition\\swinv2-large-resize-epoch=06-train_loss=0.0012-val_score=0.9991.pth'

# .pth 파일로 저장
torch.save(lit_model.state_dict(), pth_path)

In [47]:
model = Swinv2Model.from_pretrained("microsoft/swinv2-large-patch4-window12to16-192to256-22kto1k-ft")
lit_model = LitCustomModel(model)
# 체크포인트 파일 경로

ckpt_path = 'C:\\Users\\Seo\\Desktop\\2024_06_Dron_Competition\\checkpoints\\swinv2-large-resize-epoch=06-train_loss=0.0012-val_score=0.9991.ckpt'

# 체크포인트 로드
checkpoint = torch.load(ckpt_path)

# 모델 상태 로드
lit_model.load_state_dict(checkpoint['state_dict'])

lit_model.eval()

onnx_model_path = 'C:\\Users\\Seo\\Desktop\\2024_06_Dron_Competition\\swinv2-large-resize-epoch=06-train_loss=0.0012-val_score=0.9991.onnx'

input_shape = (CFG.BATCH_SIZE, CFG.CHANNELS, CFG.WIDTH, CFG.HEIGHT)

lit_model.to_onnx(onnx_model_path, input_shape, export_params=True)

TypeError: CustomModel.forward() takes from 2 to 3 positional arguments but 5 were given

In [38]:
import onnx
model = Swinv2Model.from_pretrained("microsoft/swinv2-large-patch4-window12to16-192to256-22kto1k-ft")
lit_model = LitCustomModel(model)
path_to_pytorch_model = 'C:\\Users\\Seo\\Desktop\\2024_06_Dron_Competition\\swinv2-large-resize-epoch=06-train_loss=0.0012-val_score=0.9991.pth'
lit_model = torch.load(path_to_pytorch_model)
lit_model.eval()

onnx_model_path = 'C:\\Users\\Seo\\Desktop\\2024_06_Dron_Competition\\swinv2-large-resize-epoch=06-train_loss=0.0012-val_score=0.9991.onnx'
input_shape = (CFG.BATCH_SIZE, CFG.CHANNELS, CFG.WIDTH, CFG.HEIGHT)

example_input = torch.rand(input_shape)

torch.onnx.export(lit_model, example_input, onnx_model_path, export_params=True, opset_version=11)

c:\Users\Seo\anaconda3\envs\Dron_PytorchLightning\lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [ ]:
test_df = pd.read_csv('./open/test.csv')
test_df['img_path'] = test_df['img_path'].apply(lambda x: os.path.join('./open', x))

In [ ]:
if not len(test_df) == len(os.listdir('./open/test')):
    raise ValueError()

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(size=(256,256), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

test_collate_fn = CustomCollateFn(test_transform, 'inference')
test_dataset = CustomDataset(test_df, 'img_path', mode='inference')
test_dataloader = DataLoader(test_dataset, collate_fn=test_collate_fn, batch_size=CFG.BATCH_SIZE*2)

In [ ]:
fold_preds = []
for checkpoint_path in glob('./checkpoints/swinv2-large-resize*.ckpt'):
    model = Swinv2Model.from_pretrained("microsoft/swinv2-large-patch4-window12to16-192to256-22kto1k-ft")
    lit_model = LitCustomModel.load_from_checkpoint(checkpoint_path, model=model)
    trainer = L.Trainer( accelerator='auto', precision=32)
    preds = trainer.predict(lit_model, test_dataloader)
    preds = torch.cat(preds,dim=0).detach().cpu().numpy().argmax(1)
    fold_preds.append(preds)
pred_ensemble = list(map(lambda x: np.bincount(x).argmax(),np.stack(fold_preds,axis=1)))

In [ ]:
submission = pd.read_csv('./open/sample_submission.csv')

In [ ]:
submission['label'] = le.inverse_transform(pred_ensemble)

In [ ]:
submission.to_csv('./submissions/swinv2_large_resize.csv',index=False)